# Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

import re

# First Steps

1. Get some stock market data
2. Save it in csv
3. Transform into useable format (keep as price? transform into percent change? per day?)
4. Define Neural Network Structure
5. Create Neural Network


Keeping stock prices in their standard format would allow each entry to serve as ground truth, while a percentage change would be a relative system. To find a stock change from day X to day Y would require recursively multiplying all percentage changes from day Y back to day X.

Keeping stock prices in percentage change format would help automatically account for stock splits. Adjusted price already accounts for this, but I don't want to use the adjusted price because I don't understand it enough.

## Get some stock market data

I expect the easiest way to do this is with yfinance, though I might look at some other options because yfinance isn't as well maintained as some more professional open source libraries, and it no longer links to a formal yahoo API.

Polygon.io seems promising, but the free version only has 5 API calls per minute (not sure the extent of data I can grab with a single API call, but it doesn't sound like very much), and I can only access two years of historical data.

I'll probably stick with yfinance for now. It's the easiest.

I've done some work with yfinance and one of the difficulties is that it pulls data into a **multi-index dataframe**. I'll have to **transform it into something tidy**.

I'm going to start simple and get a dataset from the DJIA (Dow Jones Industrial Average). It represents a significant portion of the market and it's a good starting point without getting too much in the weeds of big stock data with larger indexes like the S&P 500 or the Wilshire 5000.

Some obstacles to grapple with are:
- **how to handle companies which were bankrupted**. If I don't include these, then the dataset will suffer heavily from survivorship bias.
- **how to handle companies which were merged**. A merged company doesn't go out of business (and so is probably still a good investment) but **representing the change in stock price from the merge** might be difficult.

I found a list of all companies ever listed on the DJIA, but it's stored in a Wikipedia article where the changes are grouped as companies listed on the DJIA on a certain date. There is some text about which companies are moved / merged, but I'll have to parse the complete list with some programming.

There's also the issue of how far to go back for good training data. The DJIA goes back all the way to 1896, but I don't need data back that far for training. Out of convenience, let's have an 80/20 train-test split, where the training set is 40 years, and the testing set is 10 years.

I won't pull until today - that seems ridiculous. Let's set the test cutoff to be December 31, 2023. That means the first training observation will be from January 1, 1973. I don't need to pull data from before then, and I don't need to pull companies listed on the DJIA before then.

### Get company names

In [2]:
# DJIA history URL
djia_hist_comps = pd.read_html('https://en.wikipedia.org/wiki/Historical_components_of_the_Dow_Jones_Industrial_Average')

In [3]:
# combine all dataframes
djia_hist_comps_30_only = djia_hist_comps[1].iloc[:10, :]
for df in djia_hist_comps[2:25]:
    djia_hist_comps_30_only = pd.concat(
        [djia_hist_comps_30_only, df.iloc[:10, :]],
        axis='index',
        copy=False,
        ignore_index=True
    )
display(djia_hist_comps_30_only.sample(8))

,0,1,2
65,Chevron Corporation,Johnson & Johnson,United Technologies Corporation
27,The Coca-Cola Company,"Merck & Co., Inc.","Walgreens Boots Alliance, Inc."
137,Citigroup Inc.,International Business Machines Corporation,Verizon Communications Inc. ↑
155,Caterpillar Inc.,Intel Corporation ↑,The Procter & Gamble Company
179,E.I. du Pont de Nemours & Company,"Merck & Co., Inc.",F. W. Woolworth Company
216,E.I. du Pont de Nemours & Company,International Paper Company,United States Steel Corporation
3,Apple Inc.,Intel Corporation,"The Travelers Companies, Inc."
99,The Coca-Cola Company,Kraft Foods Inc.,The Walt Disney Company


In [4]:
# keeps only company name (removes arrows, subscripts, and extraneous info)
def remove_name_clutter(company_name):
    return re.sub(r"(?:\([^\(\)]*\)|\[[^\[\]]*\])|[^a-zA-Z0-9\s&]", '', company_name)

In [5]:
# convert to list of company names
djia_hist_comps_30_only = djia_hist_comps_30_only.to_numpy().flatten().tolist()

In [6]:
# get only pure company names
djia_hist_comps_30_only = [remove_name_clutter(name).strip() for name in djia_hist_comps_30_only]

In [7]:
# get distinct company names
djia_hist_comps_30_only = set(djia_hist_comps_30_only)

In [8]:
# manual removal of explicit duplicates
duplicates = [
    'UnitedHealth Group Incorporated'
]
for d in duplicates:
    djia_hist_comps_30_only.remove(d)

In [9]:
# convert to list
djia_hist_comps_30_only = sorted(list(djia_hist_comps_30_only))
djia_hist_comps_30_only[:8]

['3M Company',
 'AT&T Corporation',
 'AT&T Inc',
 'Alcoa Inc',
 'Allied Chemical Corporation',
 'AlliedSignal Incorporated',
 'Altria Group Incorporated',
 'Aluminum Company of America']

There are a few companies which are probably indistinguishably duplicates (like AT&T Corporation and AT&T Inc) but because of rebranding or mergers, the companies might be listed differently. I'll need to look into this more.

### Get company tickers

I can probably automate some of this, but I expect that I'll have to go through quite a few names in the list manually to find the tickers.

Nevermind. I've been doing some research on trying to automate this and I'm not having a lot of luck. I'll probably just manually go through the list. It'll probably take me a bit but it's all I have to go on right now, and I'll probably just force myself to find a better solution down the line for when I need to pull more companies.

In [10]:
len(djia_hist_comps_30_only)

86

In [11]:
# view company names (first half of list)
np.array(djia_hist_comps_30_only[:45])

array(['3M Company', 'AT&T Corporation', 'AT&T Inc', 'Alcoa Inc',
       'Allied Chemical Corporation', 'AlliedSignal Incorporated',
       'Altria Group Incorporated', 'Aluminum Company of America',
       'American Can Company', 'American Express Company',
       'American International Group Inc',
       'American Telephone and Telegraph Company',
       'American Tobacco Company', 'Amgen Inc',
       'Anaconda Copper Mining Company', 'Apple Inc',
       'Bank of America Corporation', 'Bethlehem Steel Corporation',
       'Caterpillar Inc', 'Chevron Corporation', 'Chrysler Corporation',
       'Cisco Systems Inc', 'Citigroup Inc', 'Dow Inc', 'DowDuPont Inc',
       'EI du Pont de Nemours & Company', 'Eastman Kodak Company',
       'Esmark Corporation', 'Exxon Corporation',
       'Exxon Mobil Corporation', 'F W Woolworth Company',
       'General Electric Company', 'General Foods Corporation',
       'General Motors Corporation', 'Goodyear Tire and Rubber Company',
       'HewlettPa

In [12]:
# company names and tickers
djia_tickers = [
    ('3M Company', 'MMM'),
    ('AT&T Inc', 'T'),
    ('Alcoa Inc', 'AA'),
    ('Honeywell International', 'HON'),
    ('Altria Group Inc', 'MO'),
    ('American Express Company', 'AXP'),
    ('American International Group Inc', 'AIG'),
    ('Amgen Inc', 'AMGN'),
    ('Apple Inc', 'AAPL'),
    ('Bank of America Corporation', 'BAC'),
    ('Caterpillar Inc', 'CAT'),
    ('Chevron Corporation', 'CVX'),
    ('Cisco Systems Inc', 'CSCO'),
    ('Citigroup Inc', 'C'),
    ('Dow Inc', 'DOW'),
    ('DuPont de Nemours Inc', 'DD'),
    ('Eastman Kodak Company', 'KODK'),
    ('Exxon Mobil Corporation', 'XOM'),
    ('General Electric Company', 'GE'),
    ('General Motors Company', 'GM'),
    ('Goodyear Tire and Rubber Company', 'GT'),
    ('HewlettPackard Company', 'HPE'),
    ('Intel Corporation', 'INTC'),
    ('International Business Machines Corporation', 'IBM'),
    ('International Paper Company', 'IP'),
    ('JPMorgan Chase Co', 'JPM')
]

Noted Changes:
- No separate tickers between 'AT&T Corporation' and 'AT&T Inc'. Several tickers for the company appear, but they look like separate divisions / asset classes, rather than the company's stock before / after the merge or rename or whatever.
- 'Allied Chemical Corporation' changed its name to 'AlliedSignal Incorporated', which then merged with and became 'Honeywell International'.
- 'Aluminum Company of America' renamed to 'Alcoa Inc'.
- 'American Can Company' merged with financial conglomerate 'Primarica Inc'.
- 'American Telephone and Telegraph Company' changed its name to 'AT&T Corporation' (no surprise there).
- 'American Tobacco Company' was restructured into a holding company called 'American Brands Inc'.
- 'Anaconda Copper Mining Company' was purchased by ARCO and then BP.
- 'Bethlehem Steel Corporation' declared bankruptcy in 2001. This is exactly the kind of company I would want to include in the bot, but alas, I'm not easily finding stock info on them.
- 'Chrysler Corporation' bankrupted and was acquired by Fiat, the US, and Canada (not sure what that means - I'm assuming the stock price reflected the bankruptcy).
- 'DowDuPont Inc' spun off (separated into?) DuPont and Dow Inc.
- 'EI du Pont de Nemours & Company' is assumedly now 'DuPont de Nemours Inc', but how it mingled with Dow, DowDuPont, etc. is beyond me right now.
- 'Esmark Corporation' is interwoven with 'JBS USA', but the two companies are too distinct to keep them properly on this list.
-  There isn't even an article for 'F W Woolworth Company', much less a ticker.
- 'Exxon Corporation', as expected, has no immediate difference from 'Exxon Mobil Corporation'.
- 'General Foods Corporation' was combined into 'Kraft Foods Inc'.
- 'Honeywell International Inc' and 'Honeywell International' are the same company.
- 'Inco Limited' was delisted and is a subsidiary of 'Vale'.
- 'International Harvester Company' was renamed to 'Navistar Inc', which is a subsidiary of 'Traton'.
- 'International Nickel Company Ltd' changed its name to 'Inco Limited' (listed above).
- 'JP Morgan & Company' is part of 'JPMorgan Chase & Co'.

A lot of the companies have merged, changed tickers, changed names, gone bankrupt, changed exchanges, etc. Which affects how easily I can access their historical data.

I've done a little research and it looks like I can probably find some data via Library of Congress (though I need to locate a public or academic library that has access to the financial records database). There's also probably newspaper databases, though I don't want to comb through those now (or at all).

I expect that I will need to account for these variables at some point, but for right now, I'm going to stick with the stocks that are easily available. It's not great for the survivorship bias (which was the whole point) but it'll be better than ignoring it entirely. Besides, this is more of an exercise in exploring how a neural network might be able to make sense of the vast interconnectedness of the stock market, rather than getting a useable stock bot immediately.

In [13]:
# view company names (second half of list)
np.array(djia_hist_comps_30_only[45:])

array(['JPMorgan Chase & Co', 'JohnsManville Corporation',
       'Johnson & Johnson', 'Kraft Foods Inc', 'McDonalds Corporation',
       'Merck & Co Inc', 'Microsoft Corporation',
       'Minnesota Mining & Manufacturing Company',
       'Navistar International Corporation', 'Nike Inc',
       'OwensIllinois Inc', 'Pfizer Inc', 'Philip Morris Companies Inc',
       'Raytheon Technologies Corporation', 'SBC Communications Inc',
       'Salesforce Inc', 'Sears Roebuck & Company',
       'Standard Oil Co of California', 'Standard Oil Co of New Jersey',
       'Swift & Company', 'Texaco Incorporated', 'The Boeing Company',
       'The CocaCola Company', 'The Goldman Sachs Group Inc',
       'The Home Depot Inc', 'The Procter & Gamble Company',
       'The Travelers Companies Inc', 'The Walt Disney Company',
       'Travelers Inc', 'USX Corporation', 'Union Carbide Corporation',
       'United Aircraft Corporation', 'United States Steel Corporation',
       'United Technologies Corporation

In [14]:
# company names and tickers (for second half of list)
djia_tickers.extend([
    ('Johnson & Johnson', 'JNJ'),
    ('McDonalds Corporation', 'MCD'),
    ('Merck & Co Inc', 'MRK'),
    ('Microsoft Corporation', 'MSFT'),
    ('Nike Inc', 'NKE'),
    ('OI Glass Inc', 'OI'),
    ('Pfizer Inc', 'PFE'),
    ('Philip Morris International Inc', 'PM'),
    ('RTX Corporation', 'RTX'),
    ('Salesforce Inc', 'CRM'),
    ('The Boeing Company', 'BOE'),
    ('The Coca-Cola Company', 'KO'),
    ('The Goldman Sachs Group Inc', 'GS'),
    ('The Home Depot Inc', 'HD'),
    ('The Procter & Gamble Company', 'PG'),
    ('The Travelers Companies Inc', 'TRV'),
    ('The Walt Disney Company', 'DIS'),
    ('United States Steel Corporation', 'X'),
    ('UnitedHealth Group Inc', 'UNH'),
    ('Verizon Communications Inc', 'VZ'),
    ('Visa Inc', 'V'),
    ('Walgreens Boots Alliance Inc', 'WBA'),
    ('Walmart Inc', 'WMT')
])

Noted Changes:
- 'Johns Manville Corporation' filed for bankruptcy and was acquired by 'Berkshire Hathaway'.
- 'Kraft Foods Inc' is a subcompany of 'Kraft Heinz'.
- 'Minnesota Mining & Manufacturing Company' changed its name to '3M Company'.
- 'Navistar' is a subsidiary (see above).
- 'OwensIllinois Inc' changed its name to 'O-I Glass Inc'.
- 'Raytheon Technologies Corporation' was renamed to 'RTX Corporation'.
- 'SBC Communications Inc' acquired and renamed to 'AT&T'.
- 'Sears Roebuck and Co' bankrupted and was acquired.
- 'Standard Oil Co of California' was renamed to 'Chevron Corporation'.
- 'Standard Oil Co of New Jersey' was renamed to 'Exxon Corporation'.
- 'Swift & Company' was renamed to 'Esmark Corporation' (see above).
- 'Texaco' is a subsidiary of 'Chevron'.
- 'Travelers Inc' was merged with Citicorp to become 'Citigroup'.
- 'USX Corporation' is the 'United States Steel Corporation'.
- 'United Technologies Corporation' merged with Raytheon to become 'Raytheon Technologies' (see above).
- 'Union Carbide Corporation' was acquired by 'Dow Chemical'.
- 'United Aircraft Corporation' was renamed to 'United Technologies Corporation'.
- 'WalMart Stores Inc' is the same as 'Walmart Inc'.
- 'Westinghouse Electric Corporation' was merged into 'CBS', then 'Viacom'.


In [15]:
len(djia_tickers)

49

Not bad, all things considered. I'll definitely have to look into **reviewing newspaper databases for historical stock info**.

Now I have to get the stock data for each ticker.

### Get data for each ticker

I found a [good article](https://www.qmr.ai/yfinance-library-the-definitive-guide/) on using yfinance.

In [16]:
# get tickers only
djia_tickers_only = [x[1] for x in djia_tickers]

In [17]:
# get tickers
tickers = yf.Tickers(djia_tickers_only)

In [18]:
# prepare dataframe
tickers_hist = tickers.history(
    start= '1972-01-01',
    end= '2022-12-31',
    auto_adjust= False
)\
.stack(level=1)\
.rename_axis(['Date','Ticker'])\
.reset_index(level=1)

[*********************100%***********************]  49 of 49 completed


In [19]:
# sort columns for readability
tickers_hist = tickers_hist[[
    'Ticker',
    'Open',
    'High',
    'Low',
    'Close',
    'Adj Close',
    'Volume',
    'Stock Splits',
    'Dividends'
]]

In [20]:
tickers_hist.head()

,Ticker,Open,High,Low,Close,Adj Close,Volume,Stock Splits,Dividends
Date,,,,,,,,,
1972-01-03,AA,4.367953,4.455563,4.355438,4.355438,1.275041,129838.0,0.0,0.0
1972-01-03,CAT,3.937500,3.979167,3.937500,3.937500,1.453360,265200.0,0.0,0.0
1972-01-03,CVX,3.585938,3.625000,3.570313,3.609375,0.557802,542400.0,0.0,0.0
1972-01-03,DIS,1.412956,1.427085,1.387266,1.394973,0.928376,1216422.0,0.0,0.0
1972-01-03,GE,7.834549,7.912738,7.818912,7.881463,1.719223,352511.0,0.0,0.0


In [21]:
tickers_hist.shape

(498498, 9)

Everything looks good here. Roughly 500k records is a solid dataset to work with. I've looked at some of the records for companies whose IPOs were after the start date (Jan 1 1973) and their first records are for their IPO. I'll have to manually **add 0-coded records for those companies whose IPOs happened after the start date**. In the meantime, this should be good to convert to store in a CSV.

## Save it in csv

In [22]:
# convert data to CSV
tickers_hist.to_csv('DJIA_price_history_base.csv')

## Transform into useable format

Now that I have some data, I need to put it into a format I can work with. The actual price probably isn't the best format because the prices are specific to each ticker. I'd like something more standardized.

My thought is to have the prices be represented as a percentage change from the previous day (or week, or month, etc). My overall goal is to use this data to try to predict whether the next day's price will reach a certain threshold.

In [23]:
# get DJIA data
djia = pd.read_csv('DJIA_price_history_base.csv')

display(djia.sample(8))

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Stock Splits,Dividends
443085,2018-06-29,INTC,49.529999,50.110001,49.500000,49.709999,42.963696,28906500.0,0.0,0.0
72161,1982-12-17,JPM,0.000000,9.222222,8.861111,9.166667,1.999012,243000.0,0.0,0.0
330913,2008-12-04,BAC,14.740000,15.360000,13.930000,14.340000,11.802994,107562100.0,0.0,0.0
149364,1991-05-20,JPM,6.250000,6.333333,6.208333,6.250000,2.374644,414600.0,0.0,0.0
271112,2003-05-14,IP,34.974373,35.002392,34.470070,34.675526,16.780722,2782539.0,0.0,0.0
200016,1996-05-28,HON,27.468960,27.588131,26.753933,27.111446,15.148894,1355823.0,0.0,0.0
63684,1981-12-01,MMM,6.781250,6.828125,6.734375,6.781250,1.844720,855200.0,0.0,0.0
99070,1986-01-15,IBM,35.731358,36.508125,35.671604,36.478249,13.843639,5830822.0,0.0,0.0


In [24]:
djia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498498 entries, 0 to 498497
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date          498498 non-null  object 
 1   Ticker        498498 non-null  object 
 2   Open          498498 non-null  float64
 3   High          498498 non-null  float64
 4   Low           498498 non-null  float64
 5   Close         498498 non-null  float64
 6   Adj Close     498498 non-null  float64
 7   Volume        498498 non-null  float64
 8   Stock Splits  498498 non-null  float64
 9   Dividends     498498 non-null  float64
dtypes: float64(8), object(2)
memory usage: 38.0+ MB


I could use the adjusted close for now, but I'll have to create something more robust in the future.

The trick now is to create a new Dataframe. I'm considering structuring it so that the companies are the columns/variables and the records are the dates, but I'm not sure how I'm going to approach constructing the tensors for the neural network with this.

First, I think I need to start by initializing zero-coded values for all companies that do not have data going back to January 1, 1972.

In [25]:
# get date for first record for all companies
companies = djia['Ticker'].unique()
listing_dates = []
for ticker in companies:
    listing_date = djia.loc[djia['Ticker'] == ticker]['Date'].min()
    listing_dates.append(listing_date)

first_dates = np.array([companies, listing_dates]).T

first_dates = pd.DataFrame(
    data= first_dates,
    columns= ['name', 'listing_date']
)


In [26]:
# get companies and first dates not listed on or prior to Jan 1, 1972
late_listers = first_dates.loc[first_dates['listing_date'] != djia['Date'].min()]
display(late_listers.sample(5))

,name,listing_date
41,CRM,2004-06-23
37,CSCO,1990-02-16
35,UNH,1984-10-17
31,HD,1981-09-22
48,DOW,2019-03-20


In [27]:
# sort all unique dates (some dates are not market days)
all_dates = djia['Date'].unique()
all_dates.sort()

# initialize data holder
entries_to_add = list()

# append zeroed entries post-1/1/1973 not yet existing for all dates for all companies
for company in late_listers['name'].values:
    listing_date = late_listers.loc[late_listers['name'] == company]['listing_date'].values[0]
    up_to_date_idx = np.where(all_dates == listing_date)[0][0]

    for date in all_dates[:up_to_date_idx]:
        zeroed_entry = [
            date,
            company,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0
        ]
        entries_to_add.append(zeroed_entry)

# construct dataframe from data holder
entries_to_add = pd.DataFrame(
    data= entries_to_add,
    columns= djia.columns
)

In [28]:
# concatenate dataframe
djia = pd.concat(
    [djia, entries_to_add],
    ignore_index= True
).sort_values(
    by= ['Ticker', 'Date'],
    ignore_index= True
)

In [32]:
# resave to price history
# djia.to_csv('DJIA_price_history.csv', index= False)

In [33]:
# get saved DJIA price history
djia = pd.read_csv('DJIA_price_history.csv')

djia.sample(8)

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Stock Splits,Dividends
520267,1994-11-25,T,15.908233,16.002644,15.861027,15.861027,2.651072,642140.0,0.0,0.0
520836,1997-02-26,T,21.242447,21.289654,20.912008,21.148035,3.918488,1976997.0,0.0,0.0
475638,2021-12-27,PFE,58.849998,59.380001,58.230000,59.200001,55.588730,27362600.0,0.0,0.0
218125,2020-11-24,GM,45.910000,46.709999,45.070000,46.459999,46.015663,20048700.0,0.0,0.0
352627,1993-03-10,KO,10.750000,10.750000,10.593750,10.718750,5.179531,4682000.0,0.0,0.0
85930,2006-09-13,BOE,26.120001,26.230000,25.980000,25.980000,4.813482,47600.0,0.0,0.0
413311,1978-11-03,MRK,1.570452,1.590331,1.550573,1.583704,0.388678,1826035.0,0.0,0.0
132231,1986-04-21,CSCO,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0


Now it's a matter of converting it into a percentage format. There are some things to keep in mind:
- There will be no change recorded if both days are zero (company is not listed)
- There will be no change recorded if the previous day is zero (company IPO, calculating percentage change would be dividing by zero)
- If using the close to calculate, stock splits or dividends will need to be checked to ensure that close prices reflect the changes properly

For now, I will use the adjusted close for proof of concept, and go from there.

In [36]:
# daily percentage change by company
djia['Percentage Change'] = djia.groupby('Ticker')['Adj Close'].pct_change() * 100

In [51]:
# fill null values
djia['Percentage Change'].fillna(value= 0.0, inplace= True)

In [54]:
# fill infinitys (DIV/0, IPO)
djia.replace(to_replace= np.inf, value= 0.0, inplace= True)

In [58]:
# check any null values in dataframe
djia.isnull().any().any()

False

Now I have a percentage change column from the previous day for all tickers.